In [ ]:
import numpy as np
import pandas as pd
from scapy.all import IP, rdpcap

In [ ]:
def resize(arr, size):
    if len(arr) >= size:
        return arr[0:size]
    return np.pad(arr,(0,size - len(arr)))

In [ ]:
data_frame = list()
# Local IP address
local_ip = '192.168.1.111' 
# Length to resize the data
resize_len = 10000  
# Path to input .pcap files
input_path = 'E:/2tab-traffic/' 
# Number of repetitions
count_repeat = 10  

# Read combinations from CSV file
combinations = pd.read_csv('E:/2taburl-label.csv')

# Loop through each combination of A and B
for index, row in combinations.iterrows():
    A, B = row['A'], row['B']
    
    for F in range(count_repeat):
        file_name = f'{input_path}{int(A)}-{int(B)}-{F}.pcap'  # Construct file name
        
        try:
            packets = rdpcap(file_name)  # Read pcap file
            data = [A, B, F]  # Initialize data list with A, B, and repetition index
            
            # Process each packet in the pcap file
            for packet in packets:
                if IP in packet:
                    ip_src = packet[IP].src
                    ip_dst = packet[IP].dst

                    # Append packet length based on source or destination IP
                    if ip_src == local_ip:
                        data.append(len(packet))
                    elif ip_dst == local_ip:
                        data.append(-len(packet))

            data_list = np.array(data)  # Convert data to numpy array
            data_list = resize(data_list, resize_len + 3)  # Resize data array
            data_list = np.array([data_list])  # Convert data list to array

            # Append to data_frame if not empty, otherwise initialize it
            if len(data_frame) == 0:
                data_frame = data_list
            else:
                data_frame = np.append(data_frame, data_list, axis=0)
        except Exception as e:
            print(f'\tError opening file {file_name}:', e)  # Print error if file can't be opened
            pass

    print(f'\rprocessing: {A}-{B}', end='')  # Print current combination being processed

print('\ndone!')

out_path = 'E:/chrome/'  # Output path
df_data = pd.DataFrame(data_frame)  # Convert data to pandas DataFrame
df_data.to_csv(out_path + 'chrome-2tab.csv', index=None, header=None)  # Save DataFrame to CSV
